In [9]:
# tf.train.Checkpoint
# 这个东西是存储权重的功能，并不是存储整个graph，存储graph需要tf.saved_model或者keras的load和save
# Checkpoints are just the weights
# 注意，在tf2里，存储和读取再也不是name-based，而是object-based

import tensorflow as tf

model_for_ck = tf.keras.Sequential()
model_for_ck.add(tf.keras.layers.Dense(2,input_shape=(None,5),name="dense"))
print(model_for_ck.trainable_variables)

model_dir="./models/ckpt_demo/read_write/"
checkpoint=tf.train.Checkpoint(model_for_ck=model_for_ck)
checkpoint.save("{}/model".format(model_dir)) # save的时候传入的参数实际上是一个prefix，真正生成的文件是./models/ckpk_demo/model.xxx
model_for_ck.trainable_variables[0].assign(tf.reshape(tf.range(0,10,dtype=tf.float32),shape=(5,2)))
print(model_for_ck.trainable_variables)
print(tf.train.list_variables("{}".format(model_dir)))
print("\n=================================================")
model_for_ck = tf.keras.Sequential()
model_for_ck.add(tf.keras.layers.Dense(2,input_shape=(None,5),name="dense1")) # name也可以不一样，文件中的模型存储的内容与代码中自变量的对应关系由checkpoint对象构造时候的dict指定
checkpoint=tf.train.Checkpoint(model_for_ck=model_for_ck)
concret_model_path=tf.train.latest_checkpoint("{}".format(model_dir))
print(concret_model_path)
checkpoint.restore(concret_model_path) # restore的时候需要传入的是具体的哪个模型
print(model_for_ck.trainable_variables)

print("\n=================================================")

another_model = tf.keras.Sequential()
another_model.add(tf.keras.layers.Dense(2,input_shape=(None,5),name="dense2"))
# name也可以不一样，文件中的模型存储的内容与代码中自变量的对应关系由checkpoint对象构造时候的dict指定
# 因为是object-based的
concret_model_path=tf.train.latest_checkpoint("{}".format(model_dir))
another_checkpoint=tf.train.Checkpoint(model_for_ck=another_model) # 这个dict传入的key是文件里记录的对象名，value指的是代码里的对象名
print(concret_model_path)
another_checkpoint.restore(concret_model_path) # restore的时候需要传入的是具体的哪个模型
print(another_model.trainable_variables)




[<tf.Variable 'dense/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[ 0.51524794, -0.12678117],
       [ 0.8410349 , -0.34851354],
       [ 0.21230781, -0.12750602],
       [-0.58230376, -0.39724493],
       [ 0.64319503,  0.11290967]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]
[<tf.Variable 'dense/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[0., 1.],
       [2., 3.],
       [4., 5.],
       [6., 7.],
       [8., 9.]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]
[('_CHECKPOINTABLE_OBJECT_GRAPH', []), ('model_for_ck/layer_with_weights-0/bias/.ATTRIBUTES/VARIABLE_VALUE', [2]), ('model_for_ck/layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE', [5, 2]), ('save_counter/.ATTRIBUTES/VARIABLE_VALUE', [])]

./models/ckpt_demo/read_write/model-1
[<tf.Variable 'dense1/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[ 0.51524794, -0.12678117],


In [10]:
# checkpoint的参数
# Checkpoint's constructor accepts keyword arguments whose values are types that contain trackable state, such as tf.keras.optimizers.Optimizer implementations, tf.Variables, tf.data.Dataset iterators, tf.keras.Layer implementations, or tf.keras.Model implementations. It saves these values with a checkpoint, and maintains a save_counter for numbering checkpoints.
print("储存=================================================")
model_for_ck_haha = tf.keras.Sequential()
model_for_ck_haha.add(tf.keras.layers.Dense(2,input_shape=(None,5),name="dense"))
checkpoint=tf.train.Checkpoint(model_for_ck_haha=model_for_ck_haha)
model_dir="./models/ckpt_demo/model_for_ck_haha/"

checkpoint.save("{}/model".format(model_dir))
print(model_for_ck_haha.trainable_variables)

print("\n读取=================================================")
another_model = tf.keras.Sequential()
another_model.add(tf.keras.layers.Dense(2,input_shape=(None,5),name="dense2"))
another_checkpoint=tf.train.Checkpoint(model_for_ck_haha=another_model)
another_checkpoint.restore(tf.train.latest_checkpoint(model_dir))
print(another_model.trainable_variables)

储存=================================================
[<tf.Variable 'dense/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[ 0.22137177,  0.15907502],
       [ 0.8041723 , -0.2964319 ],
       [ 0.90902746, -0.24875474],
       [ 0.68293464,  0.77882695],
       [ 0.875062  ,  0.8079698 ]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

读取=================================================
[<tf.Variable 'dense2/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[ 0.22137177,  0.15907502],
       [ 0.8041723 , -0.2964319 ],
       [ 0.90902746, -0.24875474],
       [ 0.68293464,  0.77882695],
       [ 0.875062  ,  0.8079698 ]], dtype=float32)>, <tf.Variable 'dense2/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]
